# メタデータを登録する
研究データのメタデータを登録するタスクです。

## メタデータ登録フォームを表示する
メタデータを登録するためのフォームを表示します。<br>
登録済みのメタデータが存在する場合は呼び出して表示します。<br>

In [ ]:
# メタデータ登録フォームを表示する
import json
import os
import traceback

import panel as pn
from IPython.display import display, clear_output
from IPython.core.display import Javascript
from requests.exceptions import RequestException

from library.task_director import TaskDirector
from library.utils.config import message as msg_config
from library.utils.widgets import Button, MessageBox
from library.utils import dg_web
from library.utils.storage_provider import grdm
#from library.utils.input import Input
from library.utils.input import get_grdm_connection_parameters
from library.utils.file import JsonFile
from typing import Union
from library.utils.config import connect as con_config
from library.utils.error import (UnusableVault, ProjectNotExist,
                                 UnauthorizedError, PermissionError)

notebook_name = 'metadata.ipynb'

class Metadata(TaskDirector):
    """メタデータを表示し、登録するクラスです。

    Attributes:
        instance:
            _form_box (pn.WidgetBox): フォームを格納する。
            _msg_output (MessageBox): ユーザーに提示するメッセージを格納する。
            schema_form (Form): フォームを作成する
            runcrate_form(dict):runcrateのフォームを作成する
            message_box(WidgetBox):メッセージボックスを格納する。
            token(str):パーソナルアクセストークン
            api_url(str):APIのURL
            project_id(str):プロジェクトID
            remote_path(str):リモート先のパス
            submit_button_title(str):ボタンのメッセージ
            submit_button(Button):ボタンの設定
    """

    def __init__(self, working_path:str) -> None:
        """Metadata コンストラクタのメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

        self.schema_form = dg_web.Form()
        self.runcrate_form = dg_web.RunCrateForm()

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        self.schema_form.form_box.width = 900
        self._form_box.append(self.schema_form.form_box)
        self.runcrate_form.form_box.width = 900
        self._form_box.append(self.runcrate_form.form_box)

        # メッセージ用ボックス
        self.message_box = pn.WidgetBox()
        self._msg_output = MessageBox()
        self._msg_output.width = 900
        self.message_box.append(self._msg_output)
        self.schema_form.msg_output.width = 900
        self.message_box.append(self.schema_form.msg_output)
        self.runcrate_form.msg_output.width = 900
        self.message_box.append(self.runcrate_form.msg_output)

        self._base_url = con_config.get('DG_WEB', 'scheme_domain')

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。
        """

        token = ""
        project_id = ""
        try:
            #token, project_id = Input.get_grdm_connection_parameters()
            token, project_id = get_grdm_connection_parameters()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except PermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        try:
            # パラメータの取得
            self.token, self.project_id = self.get_grdm_params()
            self.remote_path = dg_web.METADATA_PATH
            self.api_url = grdm.API_V2_BASE_URL
            self.schema = None
            clear_output()

            # データの取得
            if self.token and not self._msg_output.has_message():
                self.schema = self.get_schema()
                data = self.get_data(
                    token=self.token, api_url=self.api_url,
                    project_id=self.project_id, remote_path=self.remote_path
                )
                runcrates = self.get_runncrate(
                    token=self.token, api_url=self.api_url, project_id=self.project_id
                )

            # フォーム定義
            if self.schema and not self._msg_output.has_message():
                schema = self.runcrate_form.pop_schema(self.schema)
                self.schema_form.create_widgets(schema, data)
                self.runcrate_form.create_widget(runcrates, data)
                self.submit_button_title = msg_config.get('save', 'submit')
                self.submit_button = Button(width=500)
                self.submit_button.set_looks_init(self.submit_button_title)
                self.submit_button.on_click(self.submit)
                self._form_box.append(self.submit_button)
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_schema(self) -> dict:
        """jsonschemaを取得するメソッドです。

        Returns:
            dict:jsonschemaの値を返す。
        """
        schema = {}
        try:
            schema = dg_web.Api.get_metadata_schema(
                scheme_domain = self._base_url
            )
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return schema

    def get_data(self, token:str, api_url:str, project_id:str, remote_path:str) -> Union[dict,None]:
        """指定したファイルの内容を取得するメソッドです。

        Args:
            token(str):パーソナルアクセストークン
            api_url(str):APIのURL
            project_id(str):プロジェクトID
            remote_path(str:リモート先のパス

        Returns:
            Union[dict,None]:ファイルの内容を返す。
        """
        data = None
        try:
            data = grdm.GrdmMain.download_json_file(
                token=token, api_url=api_url,
                project_id=project_id, remote_path=remote_path
            )
        except FileNotFoundError:
            data = None
        except json.JSONDecodeError:
            data = {}
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        return data

    def get_runncrate(self, token:str, api_url:str, project_id:str) -> list:
        """Governed Runのindex.jsonの内容を取得するメソッドです。

        Args:
            token(str):パーソナルアクセストークン
            api_url(str):APIのURL
            project_id(str):プロジェクトID

        Returns:
            list:index.jsonの内容を返す。
        """
        crates = []
        try:
            crates = grdm.GrdmMain.download_json_file(token, api_url, project_id, dg_web.GOVRUN_INDEX_PATH)
        except (FileNotFoundError, json.JSONDecodeError):
            crates = []
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        return crates

    @TaskDirector.callback_form('メタデータを保存する')
    def submit(self, event):
        """メタデータを保存するメソッドです。"""
        file_path = os.path.join(self._abs_root_path, self.remote_path)
        tmpfile = JsonFile(file_path)
        try:
            data = self.schema_form.get_data()
            runcrate_data = self.runcrate_form.get_data()
            data.update(runcrate_data)
            tmpfile.write(data)
            self.submit_button.disabled = True
            self.submit_button.set_looks_processing(msg_config.get('save', 'doing'))
            grdm.sync(
                    token=self.token,
                    api_url=self.api_url,
                    project_id=self.project_id,
                    abs_source = file_path,
                    abs_root=self._abs_root_path
                )
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
            return
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
            return
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
            return
        finally:
            tmpfile.remove(missing_ok=True)
            self.submit_button.disabled = False
            self.submit_button.set_looks_init(self.submit_button_title)

        self._form_box.clear()
        self._msg_output.update_success(msg_config.get('dg_web', 'saved_metadata'))
        # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
        self.done_task()

Metadata(os.path.abspath('__file__')).generateFormScetion()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

notebook_name = 'metadata.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()